In [1]:
# Package import cell

import pandas as pd
import numpy as np

In [3]:
# Pull all storm events (2008-2018)
# Source URL: https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/

storms = pd.DataFrame()
years = ['2008_c20180718', '2009_c20180718', '2010_c20170726', 
         '2011_c20180718', '2012_c20170519', '2013_c20170519', 
         '2014_c20180718', '2015_c20180525', '2016_c20180718',
         '2017_c20181017', '2018_c20181116']

for year in years:
    storms = storms.append(pd.read_csv('/users/Orion/Desktop/tornado-data/StormEvents_details-ftp_v1.0_d'
                                       +str(year)+'.csv.gz'))

In [4]:
# Filter out non-tornado events

tornadoes = storms.loc[storms['EVENT_TYPE'] == 'Tornado']

# Check states where tornadoes have occurred (to help target additional data mining)

tornadoes['STATE'].unique()

array(['SOUTH CAROLINA', 'TEXAS', 'WISCONSIN', 'LOUISIANA', 'MISSOURI',
       'ILLINOIS', 'MINNESOTA', 'MISSISSIPPI', 'FLORIDA',
       'NORTH CAROLINA', 'GEORGIA', 'ALABAMA', 'SOUTH DAKOTA', 'KANSAS',
       'IOWA', 'VIRGINIA', 'COLORADO', 'TENNESSEE', 'OKLAHOMA',
       'NEBRASKA', 'WEST VIRGINIA', 'KENTUCKY', 'MARYLAND', 'ARKANSAS',
       'MONTANA', 'IDAHO', 'WYOMING', 'MICHIGAN', 'PENNSYLVANIA', 'OHIO',
       'HAWAII', 'NEW MEXICO', 'CALIFORNIA', 'ARIZONA', 'INDIANA',
       'WASHINGTON', 'NORTH DAKOTA', 'NEW YORK', 'NEW HAMPSHIRE',
       'MASSACHUSETTS', 'VERMONT', 'RHODE ISLAND', 'OREGON', 'MAINE',
       'UTAH', 'CONNECTICUT', 'NEVADA', 'NEW JERSEY', 'PUERTO RICO',
       'DELAWARE', 'DISTRICT OF COLUMBIA'], dtype=object)

In [7]:
# Pull income data (2016)
# Source URL: https://www.census.gov/data/datasets/2016/demo/saipe/2016-state-and-county.html

income = pd.read_csv('/users/Orion/Desktop/tornado-data/Median_Household_Income.csv', header=3)

# Clean income data to match 'State FIPS Code' and 'Name' formatting 
# with tornado 'STATE_FIPS' and 'CZ_NAME' data

income['State FIPS Code'] = income['State FIPS Code'].astype(int)
income = income[~income['Name'].str.contains("County") == False]
income['Name'].replace(regex=True,inplace=True,to_replace=r' County',value=r'')
income['Name'] = income['Name'].str.upper()

# Isolate 'State FIPS Code' and 'Name' (for data merging), and
# pull 'Median Household Income', which is our only column of interest

income = income[['State FIPS Code', 'Name', 'Median Household Income']]

# Merge income and tornado dataframes on FIPS and county columns (per above)

tornadoes_with_income = pd.merge(tornadoes, income, how='left', 
                                 left_on=['STATE_FIPS','CZ_NAME'], 
                                 right_on=['State FIPS Code','Name'])

In [10]:
# Pull population density data (2010)
# Source URL: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk 
# Note: URL above wasn't working for Orion -- used .CSV file emailed by Sree

density = pd.read_csv('/users/Orion/Desktop/tornado-data/Population_Density.csv', encoding='cp1252', header=1)

# Clean density data to match 'Geographic area' and Geographic area.1' formatting 
# with tornado+income 'STATE' and 'CZ_NAME' data

density = density[~density['Geographic area.1'].str.contains("County") == False]
density['Geographic area.1'].replace(regex=True,inplace=True,to_replace=r' County',value=r'')
density['Geographic area.1'] = density['Geographic area.1'].str.upper()
density['Geographic area'].replace(regex=True,inplace=True,to_replace=r'United States - ',value=r'')
density['Geographic area'] = [x.split(' -')[0] for x in density['Geographic area']]
density['Geographic area'] = density['Geographic area'].str.upper()

# Isolate 'Geographic area' and 'Geographic area.1' (for data merging), and
# pull 'Density per square mile of land area - Population' and
# 'Density per square mile of landa area - Housing units', 
# which is the only column of interest

density = density[['Geographic area', 'Geographic area.1', 'Population', 'Housing units', 
                   'Area in square miles - Total area', 'Area in square miles - Land area',
                   'Density per square mile of land area - Population', 
                   'Density per square mile of land area - Housing units']]


# Merge density and tornado+income dataframes on state and county columns (per above)

tornadoes_with_income_with_density = pd.merge(tornadoes_with_income, density, how='left',
                                              left_on=['STATE','CZ_NAME'], 
                                              right_on=['Geographic area','Geographic area.1'])

In [11]:
# Exporting merged dataset to .CSV for use in Data Cleaning notebook

tornadoes_with_income_with_density.to_csv('Merged-Tornadoes.csv') 